In [162]:
import pandas as pd
import numpy as np

In [10]:
match = pd.read_csv('./Data/Match.csv', index_col=False)
country = pd.read_csv('./Data/Country.csv', index_col=False)
team = pd.read_csv('./Data/Team.csv', index_col=False)
player = pd.read_csv('./Data/Player.csv', index_col=False)
p_att = pd.read_csv('./Data/Player_Attributes.csv', index_col=False)
league = pd.read_csv('./Data/League.csv', index_col=False)
t_att = pd.read_csv('./Data/Team_Attributes.csv', index_col=False)


In [177]:
## Add Scoring/Defensive form

alpha = 0.5 ##parameter for the decay of exp weighted mean


#Sort by earliest date first (need it in this format when applying exp average function)
match = match.sort_values(by = ['date'])


##loop over every season and every team
for season in match['season'].unique():
    df1 = match[match['season'] == season]
    for team in df1['home_team_api_id'].unique():
        df = df1[(df1['home_team_api_id'] == team)  | (df1['away_team_api_id'] == team)]
        df_home =  df1[df1['home_team_api_id'] == team]
        df_away = df1[df1['away_team_api_id'] == team]


        ##goals score/conceded in each stage of the season by the team
        goals_scored = df['home_team_goal'] * (df['home_team_api_id'] == team) +  df['away_team_goal']*(df['away_team_api_id'] == team)
        goals_conceded = df['home_team_goal'] * (df['away_team_api_id'] == team) +  df['away_team_goal']*(df['home_team_api_id'] == team)
        
        goals_scored_home = df_home['home_team_goal'] * (df_home['home_team_api_id'] == team) 
        goals_conceded_home = df_home['away_team_goal']*(df_home['home_team_api_id'] == team)
        
        goals_scored_away = df_away['away_team_goal'] * (df_away['away_team_api_id'] == team) 
        goals_conceded_away = df_away['home_team_goal']*(df_away['away_team_api_id'] == team)
        
        ##take exp weighted average
        ewm_scored = pd.Series(1).append(goals_scored[:-1].ewm(alpha = alpha).mean())
        ewd_conceded = pd.Series(1).append(goals_conceded[:-1].ewm(alpha = alpha).mean())
        ewm_scored.index = df.index
        ewd_conceded.index = df.index
        
        ewm_scored_home = pd.Series(1).append(goals_scored_home[:-1].ewm(alpha = alpha).mean())
        ewd_conceded_home = pd.Series(0.5).append(goals_conceded_home[:-1].ewm(alpha = alpha).mean())
        ewm_scored_home.index = df_home.index
        ewd_conceded_home.index = df_home.index
        
        ewm_scored_away = pd.Series(0.5).append(goals_scored_away[:-1].ewm(alpha = alpha).mean())
        ewd_conceded_away = pd.Series(1).append(goals_conceded_away[:-1].ewm(alpha = alpha).mean())
        ewm_scored_away.index = df_away.index
        ewd_conceded_away.index = df_away.index
        
        ##Add to match dataframe
        
        match.loc[df.index, 'scoring_form'] = ewm_scored
        match.loc[df.index, 'defensive_form'] = ewd_conceded
        
        match.loc[df_home.index, 'scoring_form_home'] = ewm_scored_home
        match.loc[df_home.index, 'defensive_form_home'] = ewd_conceded_home
        
        match.loc[df_away.index, 'scoring_form_away'] = ewm_scored_away
        match.loc[df_away.index, 'defensive_form_away'] = ewd_conceded_away
        
match = match.sort_index()


In [56]:
columns_keep = ['index',
'id',
'country_id',
'league_id',
'season',
'stage',
'date',
'match_api_id',
'home_team_api_id',
'away_team_api_id',
'home_team_goal',
'away_team_goal',
'home_player_1',
'home_player_2',
'home_player_3',
'home_player_4',
'home_player_5',
'home_player_6',
'home_player_7',
'home_player_8',
'home_player_9',
'home_player_10',
'home_player_11',
'away_player_1',
'away_player_2',
'away_player_3',
'away_player_4',
'away_player_5',
'away_player_6',
'away_player_7',
'away_player_8',
'away_player_9',
'away_player_10',
'away_player_11']
